In [287]:
%run ItemModel.ipynb
%run APIService.ipynb

In [11]:
import inspect, re
import pandas as pd
import csv

# Class GetData

In [288]:
class GetData:
    newest = 0
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
    APIService = APIService(newest, keyword = "thoi trang nam", categoryids = "8947")
    i = 1
    def __init__(self, newest):
        self.newest = newest
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def getData(self):
        while True:
            try:
                self.APIService.setNewest(self.newest)
                
                json_pydata = self.APIService.get()
                print(len(json_pydata))
                print(f'get! {self.i}')
                if json_pydata['items'] != None:
                    self.items_json.extend(json_pydata['items'])
                else:
                    self.parseData()
                    print('get All Data!')
                    break
                self.newest += 50
                self.i += 1
#                 self.parseData()
        
                time.sleep(10)
            except Exception as e:
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.historical_sold = item['historical_sold']
            itemModel.sold = item['sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            for tier_variation in item['tier_variations']:
                itemModel.options = len(tier_variation['options'])
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'name\tshop_location\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\trating_count\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discoun\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'sold\thistorical_sold\n')

        for item in self.all_items:
                file.write(f'{item.name}\t{item.shop_location}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.rating_count}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.sold}\t{item.historical_sold}\n')
    
    pass

# Get Data

In [289]:
getData = GetData(0)

In [290]:
getData.getData()


17
get! 1
get All Data!


In [219]:
getData.parseData()

In [286]:
getData.APIService.json_pydata

{'show_disclaimer': None,
 'query_rewrite': None,
 'adjust': None,
 'version': '453b4ce4955c7c8068c54bc30a274f21',
 'algorithm': None,
 'need_next_search': None,
 'total_count': None,
 'error': None,
 'total_ads_count': None,
 'disclaimer_infos': None,
 'nomore': None,
 'price_adjust': None,
 'json_data': None,
 'suggestion_algorithm': None,
 'items': None,
 'reserved_keyword': None,
 'hint_keywords': None}

In [284]:
getData.APIService.response.url
print('https://shopee.vn/api/v2/search_items/?by=relevancy&categoryids=8947&keyword=thoi%20trang%20nam&limit=50&newest=0&order=desc&page_type=search&skip_autocorrect=1&version=2'== 'https://shopee.vn/api/v2/search_items/?by=relevancy&categoryids=8947&keyword=thoi%20trang%20nam&limit=50&newest=0&order=desc&page_type=search&skip_autocorrect=1&version=2')

True


In [285]:
getData.APIService.url

AttributeError: 'APIService' object has no attribute 'url'

In [154]:
# Test ParseData
getData.all_items[0].name
# getData.all_items[50].name
# getData.all_items[1].name
# getData.all_items[51].name

IndexError: list index out of range

In [122]:
getData.saveFile("ShoppeItem_thoiTrangNam_aoNganTayKhongCo.tsv")

In [67]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem_thoiTrangNam_aoNganTayKhongCo.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df


,name,shop_location,shopee_verified,discount,is_adult,options,show_official_shop_label_in_title,rating_star,rating_count,flash_sale,...,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,sold,historical_sold
0,Áo Hoodies Thời Trang Dành Cho Nam (M-5Xl) 100kg),Nước ngoài,False,30%,False,7,False,4.333333,"[30, 2, 2, 2, 2, 22]",None,...,None,134,16320,30,False,False,8,0,110,113
1,Quần âu nam Hàn Quốc dáng ôm công sở quần tây nam vải co giãn nhẹ chống nhăn chống xù,Hà Nội,True,45%,False,8,False,4.595376,"[3117, 118, 76, 161, 247, 2515]",None,...,None,7252,40964,3117,False,True,8,1,2118,18198
2,Hộp 4 Sịp Đùi Boxer Thông Hơi Cao Cấp Dành Cho Nam VEIKUCOOL,Hà Nội,True,38%,False,4,False,4.851733,"[1559, 11, 5, 35, 106, 1402]",None,...,None,617,12775,1559,False,True,9,1,512,4846
3,Bộ Thể Thao Nam Chất Nỉ 4 Màu Trẻ Trung Năng Động Thời Trang Cao Cấp VICERO,Hà Nội,False,50%,False,4,True,4.885305,"[559, 5, 2, 8, 25, 519]",None,...,None,184,33373,559,False,False,9,0,1084,1366
4,"Bóp Ví dáng đứng Nam Da Bò Thật - Kiểu dáng nhỏ gọn, da Safiano cao cấp - thời trang OLUX.",TP. Hồ Chí Minh,False,20%,False,2,False,5.000000,"[13, 0, 0, 0, 0, 13]",None,...,None,1329,1896,13,False,False,8,0,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,[DEAL SOCK]Giày Nam Oxford công sở da bò / đế khâu / test lửa thoải mái,Hà Nội,False,33%,False,6,False,5.000000,"[6, 0, 0, 0, 0, 6]",None,...,None,504,90,6,False,True,9,1,0,74
1927,"Giày Thể Thao Nam Trắng Cam Tăng Chiều Cao 5cm , Sneaker cao cấp trắng cam",Hà Nội,False,16%,False,6,False,5.000000,"[4, 0, 0, 0, 0, 4]",None,...,None,6172,291,4,False,False,9,0,0,100
1928,Giày sneaker nam full trắng cực đẹp TL557 Shop Thành Long chuyên giày sneaker,TP. Hồ Chí Minh,True,49%,False,5,False,4.857143,"[14, 0, 0, 1, 0, 13]",None,...,None,17,603,14,False,False,9,0,7,33
1929,[Giày nam - Sale kịch sàn] Giày nam 2020 hàng mới về siêu chất lượng,Hà Nội,True,28%,False,6,False,4.789621,"[713, 10, 1, 25, 57, 620]",None,...,None,912,56984,713,False,False,9,0,939,1859


In [65]:
del items_df